In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

# Introduzione

Benvenuto nella guida per iniziare con il nostro progetto! Questo documento ti guiderà attraverso i passaggi fondamentali per configurare l'ambiente e iniziare a lavorare.

## Prerequisiti

Prima di iniziare, assicurati di avere i seguenti strumenti installati sul tuo sistema:

- **Node.js** (versione 14 o successiva)
- **Git** per il controllo di versione
- Un editor di testo come Visual Studio Code

[!NOTE] Se non hai ancora installato Node.js, puoi scaricarlo da [https://nodejs.org](https://nodejs.org).

## Installazione

1. Clona il repository sul tuo computer locale:
   ```bash
   git clone @@INLINE_CODE_0@@
   ```
2. Spostati nella directory del progetto:
   ```bash
   cd @@INLINE_CODE_1@@
   ```
3. Installa le dipendenze richieste:
   ```bash
   npm install
   ```

[!TIP] Usa il comando `npm ci` invece di `npm install` se vuoi garantire che le dipendenze siano installate esattamente come specificato nel file `package-lock.json`.

## Configurazione

Prima di avviare l'applicazione, è necessario configurare alcune variabili di ambiente. Crea un file `.env` nella directory principale e aggiungi le seguenti chiavi:

```env
API_KEY=@@INLINE_CODE_2@@
DATABASE_URL=@@INLINE_CODE_3@@
```

[!IMPORTANT] Non condividere mai il file `.env` o le tue chiavi API con nessuno.

## Avvio dell'applicazione

Per avviare l'applicazione in modalità di sviluppo, esegui il comando:

```bash
npm run dev
```

L'applicazione sarà disponibile all'indirizzo [http://localhost:3000](http://localhost:3000).

[!WARNING] Assicurati che nessun altro processo stia utilizzando la porta 3000.

## Test

Per eseguire i test, utilizza il comando:

```bash
npm test
```

[!CAUTION] Verifica che tutti i test passino prima di inviare una pull request.

## Conclusione

Ora sei pronto per iniziare a lavorare sul progetto. Se hai domande, consulta la nostra documentazione o contatta il team di supporto.

Buon lavoro!



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
